In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import re
from datetime import datetime, timedelta
import os
import oracledb
from sqlalchemy import create_engine

title = 'data engineer'
location = "St Louis, Missouri, United States" 
location = "Chicago, Illinois, United States"
location = "San Jose, California, United States"
#location = "United States" 
job_count = 0
exp_level = "2"  # 2==entry level
post_date = "r604800"  # r604800==Past week

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}&start={job_count}'


print(url)

https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_E=2&f_TPR=r604800&keywords="data engineer"&location=San Jose, California, United States&start=0


### FUNCTION DEFINITIONS

In [78]:
# Sends a request for one results page to LinkedIn to get a list of job titles
def get_jobs_page(job_count) -> list:
    url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}&start={job_count}'
    print("job page URL: ", url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    all_job_titles = soup.find_all("li")
    return all_job_titles

# Parses out the job title from one listing
def parse_job_id(job):
    return job.find("div", {"class":"base-card"}).get("data-entity-urn").split(":")[3]

# Get all job ids from one results page
def get_job_ids(job_count):
    job_ids = []
    all_job_titles = get_jobs_page(job_count)
    if not all_job_titles:  # if no jobs titles in the list
        print("No job titles found!")
        return None 

    all_job_ids = list(map(parse_job_id, all_job_titles))
    print("all job ids: ", all_job_ids)
    return all_job_ids
    



# I can't figure out how to get the count, the below function is useless
def get_result_count():

    base_url = "https://www.linkedin.com/jobs/search"
    url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}'
    print(url)
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    print(soup)
    count = soup.find("div", {"class":"jobs-search-results-list__subtitle"})
    print(count)




In [79]:
job_ids = []
job_count = 0
max_jobs_to_scrape = 25
# I can't get a job_count for the search query, 
# so going to scrape till it can NOT find any more job ids
# or I'm going to scrape upto 300 jobs or 12 pages
while job_count < max_jobs_to_scrape:
    page_of_ids = get_job_ids(job_count)
    if page_of_ids:
        job_ids.extend(page_of_ids)
    else:
        break
    job_count += 25
    
print(job_ids)

job page URL:  https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_E=2&f_TPR=r604800&keywords="data engineer"&location=San Jose, California, United States&start=0
all job ids:  ['3714774167', '3708157029', '3715560870', '3708150943', '3712851766', '3707550926', '3708148376', '3712609923', '3707532523', '3707507616']
['3714774167', '3708157029', '3715560870', '3708150943', '3712851766', '3707550926', '3708148376', '3712609923', '3707532523', '3707507616']


In [80]:
print(job_ids)


['3714774167', '3708157029', '3715560870', '3708150943', '3712851766', '3707550926', '3708148376', '3712609923', '3707532523', '3707507616']


In [84]:
job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
l_all_job_info = []

for id in job_ids:
    d_job_info = {}
    d_job_info["job_id"] = int(id)
    job_desc_url = job_url.format(id)
    print(job_desc_url)
    res = requests.get(job_desc_url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    # Get the company name
    d_job_info["company"] = soup.find("div", {"class":"top-card-layout__card"}).find("a").find("img").get("alt")

    # Get the location
    d_job_info["location"] = soup.find("div", {"class":"topcard__flavor-row"}).find("span", {"class":"topcard__flavor--bullet"}).text.strip()

    # Get the job title
    d_job_info["job_title"] = soup.find("h2", {"class":"top-card-layout__title"}).text.strip()

    # Get the full job description 
    d_job_info["job_description"] = soup.find("div", {"class":"show-more-less-html__markup"}).get_text(separator=u"\n")
    print(len(d_job_info["job_description"]))
    
    # Get years of experience!!!
    d_job_info["experience"] = re.findall(r".*\D\d{1,2}\D.*years?", d_job_info["job_description"])
    d_job_info["experience"] = "\n".join(d_job_info["experience"])

    # Get Seniority level, Employment type, Job function, Industries
    job_criteria_list = soup.find("ul", {"class":"description__job-criteria-list"}).find_all("li")
    for criteria in job_criteria_list:
        criteria = criteria.text.split("\n") # convert lines to a list
        criteria = [i.strip() for i in criteria if i.strip()] # remove lines with only white space
        criteria[0] = criteria[0].replace(" ", "_").lower()
        d_job_info.update({criteria[0]:criteria[1]})

    # Get job posting date
    posting_date = soup.find("span", {"class":"posted-time-ago__text"}).text.strip()
    posting_num = int(re.match(r'\d{1,2}',posting_date).group())
    if "minute" in posting_date:
        d_job_info["posting_date"] = datetime.today() - timedelta(minutes=posting_num)
    elif "hour" in posting_date:
        d_job_info["posting_date"] = datetime.today() - timedelta(hours=posting_num)
    elif "day" in posting_date:
        d_job_info["posting_date"] = datetime.today() - timedelta(days=posting_num)
    else:
        d_job_info["posting_date"] = ""
    
    
    # !!! Get Other useful info with AI !!!

    
    # Get URL 
    d_job_info["url"] = soup.find("a", {"class":"topcard__link"}).get("href")
    print(d_job_info["url"])

    # Append the job info (dict) to the list of job info
    l_all_job_info.append(d_job_info)

# Convert list of all job info dicts to a dataframe
df_all_job_info = pd.DataFrame(l_all_job_info)
df_all_job_info


https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3714774167
8147
https://www.linkedin.com/jobs/view/research-data-engineer-at-linkedin-3714774167?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3708157029
1212
https://www.linkedin.com/jobs/view/azure-sql-developer-data-engineer-at-rit-solutions-inc-3708157029?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3715560870
2189
https://www.linkedin.com/jobs/view/remote-azure-data-engineer-with-data-bricks-at-techfetch-com-on-demand-tech-workforce-hiring-platform-3715560870?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3708150943
2078
https://www.linkedin.com/jobs/view/azure-data-engineer-with-data-bricks-at-tanisha-systems-inc-3708150943?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3712851766
1873
https://www.linkedin.com/jobs/view/video-data-collector-at-spectraforce-3712851766?t

,job_id,company,location,job_title,job_description,experience,seniority_level,employment_type,job_function,industries,posting_date,url
0,3714774167,LinkedIn,San Francisco Bay Area,Research Data Engineer,\n LinkedIn is the world’s largest prof...,BA/BS degree or equivalent experience \n3+ yea...,Entry level,Full-time,Information Technology,IT Services and IT Consulting,2023-09-08 04:39:36.995642,https://www.linkedin.com/jobs/view/research-da...
1,3708157029,"RIT Solutions, Inc.","Sunnyvale, CA",Azure SQL Developer/Data Engineer,\n\nTitle: Azure SQL Developer/Data Engineer\n...,1 plus years\n 5 plus years,Entry level,Contract,Information Technology,Staffing and Recruiting,2023-09-08 04:39:37.746336,https://www.linkedin.com/jobs/view/azure-sql-d...
2,3715560870,TechFetch.com - On Demand Tech Workforce hirin...,"San Jose, CA",Remote - Azure Data Engineer with Data Bricks,"\n ""ALL our jobs are US based and candi...","""ALL our jobs are US based and candida...",Entry level,Part-time,Information Technology,IT Services and IT Consulting,2023-09-09 04:39:38.138934,https://www.linkedin.com/jobs/view/remote-azur...
3,3708150943,"Tanisha Systems, Inc","San Jose, CA",Azure Data Engineer with Data Bricks,\n ob role: \nAzure Data Engineer with ...,Required Skills/Qualifications\n6+ years\nDesi...,Entry level,Contract,Information Technology,Human Resources Services,2023-09-08 04:39:38.722594,https://www.linkedin.com/jobs/view/azure-data-...
4,3712851766,SPECTRAFORCE,"Cupertino, CA",Video Data Collector,\n\nJob Title: Video Data Collector\nDuration:...,,Entry level,Contract,Art/Creative and Writing/Editing,Staffing and Recruiting,2023-09-09 04:39:39.023121,https://www.linkedin.com/jobs/view/video-data-...
5,3707550926,"TalentBurst, an Inc 5000 company","Cupertino, CA",Video Data Collector #: 23-15206,\n\nJob Description\n Title: Video Data Collec...,,Entry level,Contract,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-09 04:39:39.328467,https://www.linkedin.com/jobs/view/video-data-...
6,3708148376,Russell Tobin,"Cupertino, CA",Process Technician I,\n\nTitle: Video Data Collector\nJob Type: 6 m...,,Entry level,Contract,Management and Manufacturing,Staffing and Recruiting,2023-09-08 04:39:39.735684,https://www.linkedin.com/jobs/view/process-tec...
7,3712609923,SPECTRAFORCE,"Cupertino, CA",Video Data Collector,\n\nJob Title: Video Data Collector\nDuration:...,,Entry level,Full-time,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-07 04:39:40.045584,https://www.linkedin.com/jobs/view/video-data-...
8,3707532523,"TalentBurst, an Inc 5000 company","Cupertino, CA",Video Data Collector,\n\nTitle: Video Data Collector\n \n Location:...,,Entry level,Contract,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-08 04:39:40.350584,https://www.linkedin.com/jobs/view/video-data-...
9,3707507616,ICONMA,"Cupertino, CA",Process Technician I,\n\nProcess Technician I\n \n \n \n \n Loca...,,Entry level,Contract,Management and Manufacturing,Staffing and Recruiting,2023-09-08 04:39:40.707149,https://www.linkedin.com/jobs/view/process-tec...


### Get all the job IDs that already exist in the database

In [89]:
cs = '''(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.us-sanjose-1.oraclecloud.com))(connect_data=(service_name=ga3e236c6957ba6_oltpdb_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))'''
user="appuser"
password=os.environ['ORACLE_PASSWORD_APPUSER']
engine = create_engine(
    f'oracle+oracledb://{user}:{password}@{cs}'
    )

with engine.connect() as conn:
    df_jobs_exists = pd.read_sql_query(f'SELECT job_id FROM tbl_jobs WHERE job_id IN ({",".join(job_ids)})', conn)
               
df_jobs_exists

,job_id
0,3708157029
1,3714774167


### Filter out all the IDs that exist in the database

In [90]:
df_all_job_info = df_all_job_info[~df_all_job_info['job_id'].isin(list(df_jobs_exists['job_id']))]
df_all_job_info

,job_id,company,location,job_title,job_description,experience,seniority_level,employment_type,job_function,industries,posting_date,url
2,3715560870,TechFetch.com - On Demand Tech Workforce hirin...,"San Jose, CA",Remote - Azure Data Engineer with Data Bricks,"\n ""ALL our jobs are US based and candi...","""ALL our jobs are US based and candida...",Entry level,Part-time,Information Technology,IT Services and IT Consulting,2023-09-09 04:39:38.138934,https://www.linkedin.com/jobs/view/remote-azur...
3,3708150943,"Tanisha Systems, Inc","San Jose, CA",Azure Data Engineer with Data Bricks,\n ob role: \nAzure Data Engineer with ...,Required Skills/Qualifications\n6+ years\nDesi...,Entry level,Contract,Information Technology,Human Resources Services,2023-09-08 04:39:38.722594,https://www.linkedin.com/jobs/view/azure-data-...
4,3712851766,SPECTRAFORCE,"Cupertino, CA",Video Data Collector,\n\nJob Title: Video Data Collector\nDuration:...,,Entry level,Contract,Art/Creative and Writing/Editing,Staffing and Recruiting,2023-09-09 04:39:39.023121,https://www.linkedin.com/jobs/view/video-data-...
5,3707550926,"TalentBurst, an Inc 5000 company","Cupertino, CA",Video Data Collector #: 23-15206,\n\nJob Description\n Title: Video Data Collec...,,Entry level,Contract,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-09 04:39:39.328467,https://www.linkedin.com/jobs/view/video-data-...
6,3708148376,Russell Tobin,"Cupertino, CA",Process Technician I,\n\nTitle: Video Data Collector\nJob Type: 6 m...,,Entry level,Contract,Management and Manufacturing,Staffing and Recruiting,2023-09-08 04:39:39.735684,https://www.linkedin.com/jobs/view/process-tec...
7,3712609923,SPECTRAFORCE,"Cupertino, CA",Video Data Collector,\n\nJob Title: Video Data Collector\nDuration:...,,Entry level,Full-time,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-07 04:39:40.045584,https://www.linkedin.com/jobs/view/video-data-...
8,3707532523,"TalentBurst, an Inc 5000 company","Cupertino, CA",Video Data Collector,\n\nTitle: Video Data Collector\n \n Location:...,,Entry level,Contract,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-08 04:39:40.350584,https://www.linkedin.com/jobs/view/video-data-...
9,3707507616,ICONMA,"Cupertino, CA",Process Technician I,\n\nProcess Technician I\n \n \n \n \n Loca...,,Entry level,Contract,Management and Manufacturing,Staffing and Recruiting,2023-09-08 04:39:40.707149,https://www.linkedin.com/jobs/view/process-tec...


### Insert job info (dataframe) into the database

In [91]:
df_all_job_info.to_sql('tbl_jobs', engine, 'appuser', if_exists='append', index=False, method=None)

-1

### Double Check database insert

In [92]:
with engine.connect() as conn:
    df_check_database = pd.read_sql_query(f'SELECT job_id FROM tbl_jobs WHERE job_id IN ({",".join(job_ids)})', conn)

df_check_database

,job_id
0,3707507616
1,3707532523
2,3707550926
3,3708148376
4,3708150943
5,3708157029
6,3712609923
7,3712851766
8,3714774167
9,3715560870
